In [7]:
import time

# getpass - to input our password without showing it in the notebook
from getpass import getpass

# Juicy stuff - these are the Classes we will use for interaction with a webpage:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common import NoSuchElementException, ElementClickInterceptedException

# libraries for interacting with the operating system (OS)
import pathlib
import os
from os.path import join

import pandas as pd
import random
import re
from bs4 import BeautifulSoup
import re

import warnings
warnings.filterwarnings('ignore')

import numpy as np


In [8]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


In [9]:
def perform_search_and_scroll(driver):
    try:
        max_attempts = 10
        attempts = 0
        while attempts < max_attempts:
            try:
                button_search = driver.find_element(By.CLASS_NAME, 'zoeken-resultaten-lijst-meer')
                button_search.click()
            except:
                attempts += 1

        # Scroll to the end of the page
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)

        # Optionally, you can add a delay to give the page time to load new content
        # import time
        # time.sleep(2)  # 2 seconds of delay (adjust as needed)
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [11]:
def extract_school_details (url):
    driver.get(url)
    
    #cookies
    import pickle
    cookies_dir = 'saved_cookies'
    lis_dir = os.listdir()

    if cookies_dir not in lis_dir:
        os.mkdir(cookies_dir)
    else:
        pass # os.removedirs(cookies_dir) --> to remove a directory

    save_location = cookies_dir + '/cookies.pkl'
    pickle.dump(driver.get_cookies() , open(save_location,"wb"))
    
    max_attempts = 10 
    attempts = 0

    while attempts < max_attempts:
        try:
            button_search = driver.find_element(By.CLASS_NAME, 'zoeken-resultaten-lijst-meer')
            button_search.click()
               
        except:
            attempts += 1
               
    
    #getting data
    html = driver.page_source
    soup = BeautifulSoup(html)
    soup.find_all(attrs={'class': re.compile(r'zoekresultaat-title')})
    
    #links
    school_links_dirty = soup.select('h3 a')
    school_links_clean = [item.get('href') for item in school_links_dirty]
    
    #names
    school_list_dirty = soup.find_all(attrs={'class': re.compile(r'zoekresultaat-title')})
    school_list_clean = [school.text.strip() for school in school_list_dirty]
    
    #adresses
    adress_list_dirty = soup.find_all(attrs={'class': re.compile(r'zoekresultaat-adres')})
    adress_list_clean = [job.text.strip() for job in adress_list_dirty]
    
    #details
    details_list_dirty = soup.find_all(attrs={'class': re.compile(r'zoekresultaat-details')})
    details_list_clean = [job.text.strip() for job in details_list_dirty]
    
    #number of students
    details_list_clean
    pattern = r'\d+'
    number_of_students = [re.search(pattern, item).group() if re.search(pattern, item) else '' for item in details_list_clean]
        
    return pd.DataFrame({'name':school_list_clean, 'address':adress_list_clean, 'link':school_links_clean, 'details':details_list_clean, 'n_students':number_of_students})

In [12]:
def create_database (csv):    
    mschool_links = pd.read_csv(csv)
    dfs_list = []
    for ind in mschool_links.index:
        df = (extract_school_details(mschool_links['Link'][ind]))
        df['neighborhood'] = mschool_links['Name'][ind]
        dfs_list.append(df)
    final_df = pd.concat(dfs_list, ignore_index=True)
    return final_df

In [13]:
final_mschool_df = create_database('places_middle_schools.csv')

In [14]:
final_mschool_df['type'] = 'Middle School'
final_mschool_df

,name,address,link,details,n_students,neighborhood,type
0,Jozefschool Katholiek Basisonderwijs,Gerberastraat 2,/basisscholen/aalsmeer/8825/jozefschool-kathol...,KatholiekBasisonderwijs436 leerlingen,436,Aalsmeer,Middle School
1,Rooms Katholieke Basisschool Oosteinder,Catharina-Amalialaan 66 B,/basisscholen/aalsmeer/8827/rooms-katholieke-b...,KatholiekBasisonderwijs373 leerlingen,373,Aalsmeer,Middle School
2,IKC Triade,Dreef 1A,/basisscholen/aalsmeer/24314/ikc-triade/,Prot. christelijkBasisonderwijs376 leerlingen,376,Aalsmeer,Middle School
3,De Brug,Catharina-Amalialaan 66 D,/basisscholen/aalsmeer/24358/de-brug/,Prot. christelijkBasisonderwijs173 leerlingen,173,Aalsmeer,Middle School
4,Openbare Basisschool Samen Een,Schoolstraat 15,/basisscholen/aalsmeer/6586/openbare-basisscho...,OpenbaarBasisonderwijs188 leerlingen,188,Aalsmeer,Middle School
...,...,...,...,...,...,...,...
5525,Buitenwijs,Nilantsweg 77,/basisscholen/zwolle/26866/buitenwijs/,OverigeBasisonderwijs,,Zwolle,Middle School
5526,Kentalis Enkschool SO Zwolle,Jan Buschstraat 6,/basisscholen/zwolle/23943/kentalis-enkschool-...,Algemeen bijzonderSpeciaal onderwijs158 leerli...,158,Zwolle,Middle School
5527,Michaelschool,Bachlaan 8,/basisscholen/zwolle/8415/michaelschool/,Algemeen bijzonderBasisonderwijs212 leerlingen...,212,Zwolle,Middle School
5528,Ridder Jorisschool,Korianderplein\t\t 8-10,/basisscholen/zwolle/24895/ridder-jorisschool/,Algemeen bijzonderBasisonderwijs187 leerlingen...,187,Zwolle,Middle School


In [21]:
final_bschool_df = create_database('places_basic_schools.csv')

In [22]:
final_bschool_df['type'] = 'Basic School'
final_bschool_df

,name,address,link,details,n_students,neighborhood,type
0,Kindcentrum De Wegwijzer,Harm Tiesingstraat 74,/basisscholen/2e-exloërmond/25159/kindcentrum-...,Prot. christelijkBasisonderwijs84 leerlingen,84,2e Exloërmond,Basic School
1,Openbare Basisschool De Westhoek,Schoolstraat 27,/basisscholen/2e-exloërmond/7595/openbare-basi...,OpenbaarBasisonderwijs44 leerlingen,44,2e Exloërmond,Basic School
2,Basisschool De Tweeklank,Albardastraat 35,/basisscholen/aadorp/11785/basisschool-de-twee...,SamenwerkingBasisonderwijs116 leerlingenDalton,116,Aadorp,Basic School
3,JHR Willem Versluijsschool,Prelaatweg 22,/basisscholen/aagtekerke/25963/jhr-willem-vers...,Prot. christelijkBasisonderwijs191 leerlingen,191,Aagtekerke,Basic School
4,Christelijke Basisschool de Schutse,Paardelandsdrift 9,/basisscholen/aalden/9661/christelijke-basissc...,Prot. christelijkBasisonderwijs62 leerlingen,62,Aalden,Basic School
...,...,...,...,...,...,...,...
6960,Buitenwijs,Nilantsweg 77,/basisscholen/zwolle/26866/buitenwijs/,OverigeBasisonderwijs,,Zwolle,Basic School
6961,Kentalis Enkschool SO Zwolle,Jan Buschstraat 6,/basisscholen/zwolle/23943/kentalis-enkschool-...,Algemeen bijzonderSpeciaal onderwijs158 leerli...,158,Zwolle,Basic School
6962,Michaelschool,Bachlaan 8,/basisscholen/zwolle/8415/michaelschool/,Algemeen bijzonderBasisonderwijs212 leerlingen...,212,Zwolle,Basic School
6963,Ridder Jorisschool,Korianderplein\t\t 8-10,/basisscholen/zwolle/24895/ridder-jorisschool/,Algemeen bijzonderBasisonderwijs187 leerlingen...,187,Zwolle,Basic School


In [23]:
df_final = pd.concat([final_mschool_df,final_bschool_df], ignore_index=True)
df_final

,name,address,link,details,n_students,neighborhood,type
0,Jozefschool Katholiek Basisonderwijs,Gerberastraat 2,/basisscholen/aalsmeer/8825/jozefschool-kathol...,KatholiekBasisonderwijs436 leerlingen,436,Aalsmeer,Middle School
1,Rooms Katholieke Basisschool Oosteinder,Catharina-Amalialaan 66 B,/basisscholen/aalsmeer/8827/rooms-katholieke-b...,KatholiekBasisonderwijs373 leerlingen,373,Aalsmeer,Middle School
2,IKC Triade,Dreef 1A,/basisscholen/aalsmeer/24314/ikc-triade/,Prot. christelijkBasisonderwijs376 leerlingen,376,Aalsmeer,Middle School
3,De Brug,Catharina-Amalialaan 66 D,/basisscholen/aalsmeer/24358/de-brug/,Prot. christelijkBasisonderwijs173 leerlingen,173,Aalsmeer,Middle School
4,Openbare Basisschool Samen Een,Schoolstraat 15,/basisscholen/aalsmeer/6586/openbare-basisscho...,OpenbaarBasisonderwijs188 leerlingen,188,Aalsmeer,Middle School
...,...,...,...,...,...,...,...
12490,Buitenwijs,Nilantsweg 77,/basisscholen/zwolle/26866/buitenwijs/,OverigeBasisonderwijs,,Zwolle,Basic School
12491,Kentalis Enkschool SO Zwolle,Jan Buschstraat 6,/basisscholen/zwolle/23943/kentalis-enkschool-...,Algemeen bijzonderSpeciaal onderwijs158 leerli...,158,Zwolle,Basic School
12492,Michaelschool,Bachlaan 8,/basisscholen/zwolle/8415/michaelschool/,Algemeen bijzonderBasisonderwijs212 leerlingen...,212,Zwolle,Basic School
12493,Ridder Jorisschool,Korianderplein\t\t 8-10,/basisscholen/zwolle/24895/ridder-jorisschool/,Algemeen bijzonderBasisonderwijs187 leerlingen...,187,Zwolle,Basic School


In [24]:
df_final.to_csv('final_allschools_df.csv')